In [1]:
# Convolutional Network for Image Classification
# Michael Timbes

In [2]:
import tensorflow as tf
import imageprep

In [3]:
train_path = 'NotFace'
test_path = 'test_set'

IMAG_X = 100

# Keys
keyA = 'face' #True Class
keyB = 'notface' #False Class

In [4]:
# Data Import
training_images, training_labels, training_original = imageprep.ImportImages(train_path, IMAG_X, IMAG_X, keyA, keyB)

# Data Convert
training_images = imageprep.shape_up_X(training_images,IMAG_X)
training_images.astype(float);
print(training_images.shape)

(123, 100, 100, 1)


In [7]:
# Layer 1

W_conv1 = imageprep.weight_variable([5, 5, 1, 32])
b_conv1 = imageprep.bias_variable([32])

# Convolution of Layer 1
h_conv1 = tf.nn.relu(imageprep.conv2d(training_images, W_conv1) + b_conv1)
h_pool1 = imageprep.max_pool_2x2(h_conv1)

In [8]:
# Connected Layer
W_fc1 = imageprep.weight_variable([50 * 50 * 32, 64])
b_fc1 = imageprep.bias_variable([64])

h_pool2_flat = tf.reshape(h_pool1, [-1, 50* 50* 32])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [9]:
# Output Layer
W_fc2 = imageprep.weight_variable([64, 2])
b_fc2 = imageprep.bias_variable([2])

y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2

In [ ]:
# Set Up Training
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))